In [1]:
#//----------------------------
#//LIBRARIES
    #Math
import math
    #Numeric Python
import numpy as np
    #Pandas (dataframes)
import pandas as pd
    #datetime for fate manipulation
from datetime import date, datetime, timedelta  
    #Regex for advanced string matching
import re
    #for time related stuff
import time
    #json library
import json
    #Analyst tools
import sys
sys.path.append('../')
from analysts_tools.growth import *
    #Procurement tools
from procurement_lib import send_slack_notification
from procurement_lib import redash
from analysts_tools.redash_methods import *

import random
import time
import datetime

from sklearn.model_selection import train_test_split 

In [2]:
#//---------------------------------------------------------------------------------------------------------------------------
#// Parametros

city_code = 'CMX'
    
#year-month-day
todays_date = datetime.datetime.today().strftime('%Y-%m-%d')
#non_buyers_date = '2022-08-26' #1 months
customer_date_desde = (datetime.datetime.today() - datetime.timedelta(days=20)).strftime('%Y-%m-%d') #1 months
    #data for sku sales data
#sku_date_desde = '2022-06-26' #2 months

dias = 1

tasas = {'BOG': 3776,
         'BAQ': 3776,
         'MDE': 3776,
         'CMX': 19.65,
         'GDL': 19.65,
         'PBC': 19.65,
         'SPO': 4.75,
         'BHZ': 4.75,
         'CWB': 4.75,
         'VCP': 4.75}

t_cambio = tasas[city_code]

In [3]:
clear_buyers = get_fresh_query_result("https://internal-redash.federate.frubana.com/",136016,'SeoGHWmDUaaBi7VXje1s9zYNiMD1VHQ1K1DYOxiF',{},20)

200


In [4]:
clear_buyers["last_order_date"] = pd.to_datetime(clear_buyers["last_order_date"]) 
clear_buyers = clear_buyers[clear_buyers.last_order_date >= customer_date_desde].copy()
clear_buyers.head()

,customer_id,last_order_date,recency_days,registered_city
0,308238668,2023-02-28,0,BAQ
1,307548193,2023-02-27,1,SPO
2,307574923,2023-02-27,1,CWB
3,307577156,2023-02-27,1,BOG
4,307508893,2023-02-27,1,BAQ


In [24]:
query = """
WITH
info AS (
SELECT DISTINCT
    s.site_identifier_value as region_code,
    --ffg.close_date,
    bo.customer_id,
    --bs.addl_product_id as padre_sku_id,
    --bs2.name AS card_name,
    bcat2.name AS category,
    --bcat.name AS subcat,
    COUNT(DISTINCT EXTRACT(WEEK FROM ffg.close_date)) AS count_weeks,
    --baid.adjustment_reason,
    SUM ( (boi.quantity * foi.step_unit * boi.sale_price)/{tipo_cambio}) AS gmv_usd
    --SUM ( COALESCE((baid.adjustment_value * boip.quantity * foi.step_unit)/{tipo_cambio}, 0) ) AS discount_applied,
    --SUM ( boi.quantity* foi.step_unit ) AS cant

FROM postgres_broadleaf_federate."broadleaf.blc_fulfillment_group_item"     bfgi
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_group"    bfg        ON bfgi.fulfillment_group_id = bfg.fulfillment_group_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_fulfillment_group"     ffg        ON ffg.fulfillment_group_id = bfg.fulfillment_group_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order"                bo         ON bo.order_id = bfg.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_site"                 s          ON bo.site_disc = s.site_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order_item"           boi        ON bfgi.order_item_id= boi.order_item_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order_item"            foi        ON boi.order_item_id= foi.order_item_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order"                 fo         ON fo.order_id = bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_order"    bfo        ON bfo.fulfillment_group_id= bfg.fulfillment_group_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_discrete_order_item"  bdoi       ON bdoi.order_item_id = boi.order_item_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_sku"                  bs         ON bs.sku_id = bdoi.sku_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_product"              bp         ON bs.addl_product_id = bp.product_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_sku"                  bs2        ON bs2.sku_id = bp.default_sku_id --Conectar la tarjeta con el sku que guarda la info
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_category"             bcat       ON bcat.category_id = bp.default_category_id
LEFT JOIN  postgres_broadleaf_federate."broadleaf.blc_order_payment"        bop        ON bop.order_id = bo.order_id
LEFT JOIN  postgres_broadleaf_federate."broadleaf.fb_order_type"            fot        ON fot.fb_order_type_id=fo.fb_order_type_id
LEFT JOIN  postgres_broadleaf_federate."broadleaf.blc_category_xref"        bcx        ON bcx.sub_category_id = bp.default_category_id AND bcx.archived='N' AND bcx.sndbx_tier is NULL
LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_category"              bcat2      ON bcx.category_id = bcat2.category_id
--LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_order_item_price_dtl"  boip       ON  boi.order_item_id = boip.order_item_id
--LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_order_item_dtl_adj"    baid       ON  baid.order_item_price_dtl_id = boip.order_item_price_dtl_id

WHERE 
   DATE(ffg.close_date) >= current_date - interval '6 week' 
   AND DATE(ffg.close_date) <=  current_date
   AND bo.order_status='SUBMITTED'
   AND fo.fb_order_status_id IN (1,6,7,8) 
   AND s.site_identifier_value = '{ciudad}'
   AND bfo.status NOT IN ('ARCHIVED','CANCELLED')
   AND bop.archived = 'N'
   AND (fot.name IS NULL OR fot.name <> 'REFUND')
   -- FIX SUPER DESCUENTOS
   AND bcat2.category_id not in ('110873','-1000','100768','100765','100815') --ids de super descuentos en cada país
   AND bcat.name <> 'Oferton Frubana' AND bcat2.name <> 'Oferton Frubana'

GROUP BY 1,2,3--,5,6,7
)

SELECT 
*
FROM info
--WHERE count_weeks > 5
""".format(tipo_cambio=t_cambio, ciudad=city_code)

data_ventas_aux = read_connection_data_warehouse.runQuery(query)
data_ventas_aux[["gmv_usd"]] = data_ventas_aux[["gmv_usd"]].astype(float)
#data_ventas["submit_date"] = pd.to_datetime(data_ventas["submit_date"])

In [25]:
data_ventas_aux.head()

,region_code,customer_id,category,count_weeks,gmv_usd
0,CMX,187752957,Lácteos & Huevos,1,66.37857
1,CMX,51774044,Frutas & Verduras,3,271.77201
2,CMX,263791339,Frutas & Verduras,2,51.92516
3,CMX,52918709,Desechables,1,48.56996
4,CMX,293750489,Lácteos & Huevos,3,4.90835


In [26]:
data_ventas_aux.count_weeks.max()

7

In [27]:
cat = ['Frutas & Verduras']

In [37]:
data_ventas = data_ventas_aux[(data_ventas_aux.category.isin(cat)) & 
                              (data_ventas_aux.customer_id.isin(clear_buyers.customer_id.unique()))].copy()

In [38]:
f = data_ventas[data_ventas.count_weeks>=6].groupby("category", as_index=False).agg({'customer_id':[lambda x: x.nunique(),lambda x: list(x.unique())]}).droplevel(0, axis="columns")
f.columns = ['category', 'customers', 'list_customers']
f

,category,customers,list_customers
0,Frutas & Verduras,1409,"[16336605, 234957383, 147884243, 288145499, 14..."


In [39]:
def create_groups(subcat):
    s = f[f.category == subcat].list_customers.to_numpy()[0]
    
    d = data_ventas[(data_ventas.customer_id.isin(s)) & (data_ventas.category == subcat)].groupby("customer_id").gmv_usd.mean().reset_index()
    
    customer_control, customer_test = train_test_split(s, test_size = 0.50)
    
    alpha=0.01
    aux=0
    
    while abs(1-(d[d.customer_id.isin(customer_control)]['gmv_usd'].mean()/d[d.customer_id.isin(customer_test)]['gmv_usd'].mean())) > alpha:
        customer_control, customer_test = train_test_split(s, test_size = 0.50)
        
        aux+=1
        if aux == 1000:
            alpha+=0.01
            aux=0
        
    all_users = clear_buyers[(~clear_buyers.customer_id.isin(customer_test)) & (clear_buyers.registered_city == city_code)].customer_id.unique().tolist()
    customer_control = [int(customer) for customer in customer_control]
    customer_test = [int(customer) for customer in customer_test]
    
    segt_offer = {'name':"DAY_Elasticidad"+todays_date+"_"+city_code+"_"+subcat+"_ALL_D1", 'customersIds': list(all_users)}
    segt_control = {'name':"DAY_Elasticidad"+todays_date+"_"+city_code+"_"+subcat+"_CONTROL_D1", 'customersIds': list(customer_control)}
    segt_test = {'name':"DAY_Elasticidad"+todays_date+"_"+city_code+"_"+subcat+"_TEST_D1", 'customersIds': list(customer_test)}
    
    print("SUBCAT:",subcat, "// TOTAL_USERS:", len(all_users), "// TEST:",len(customer_test), "// CONTROL:",len(customer_control))
    
    return customer_control, customer_test, segt_test, segt_control, segt_offer

In [40]:
global_segment_list = []
r= {}

for category in f.category:
    control, test, segment_t, segment_c, segment_o = create_groups(category)
    
    #r['CONTROL_'+todays_date+"_"+city_code+"_"+subcat] = control
    #r['TEST_'+todays_date+"_"+city_code+"_"+subcat] = test
    
    global_segment_list.append(segment_t)
    global_segment_list.append(segment_c)
    global_segment_list.append(segment_o)

df_test_control = pd.DataFrame.from_dict(r, orient='index').T

SUBCAT: Frutas & Verduras // TOTAL_USERS: 8998 // TEST: 705 // CONTROL: 704


In [41]:
len(clear_buyers[clear_buyers.registered_city == city_code])

9699

In [42]:
def crear_seg(global_value):
  import requests
  import json

  url = f"https://{city_code}.frubana.com/api/v1/segments/create?api-key=00_growth-team-zzz-qqq_001"

  payload = json.dumps(global_value)


  headers = {
    'Cookie': 'AWSALB=WGKrKvA1CFOY0m92xSENGAseuE4LrcfQh9y8jK/loGBJdizNm+FvjOqUqVghxdfLkH6Jsa9sfWft2NELchGAP/LNZMXMYhrOE70qDnUuOHATpdJj1UZBMEUBSIN1; AWSALBCORS=WGKrKvA1CFOY0m92xSENGAseuE4LrcfQh9y8jK/loGBJdizNm+FvjOqUqVghxdfLkH6Jsa9sfWft2NELchGAP/LNZMXMYhrOE70qDnUuOHATpdJj1UZBMEUBSIN1; AWSALB=jLY02FPysvF1vTrKLhXbk3ibzmyiAqFJ1ezYVu5PjodkU2QmPHDpRJhb9CvOlOyQ1Ix0mJR1ig+eH9zwJ9i5EOtgsgx79NSROogQb2Ua6yLX/I3bJJ6xtcVO3puI; AWSALBCORS=jLY02FPysvF1vTrKLhXbk3ibzmyiAqFJ1ezYVu5PjodkU2QmPHDpRJhb9CvOlOyQ1Ix0mJR1ig+eH9zwJ9i5EOtgsgx79NSROogQb2Ua6yLX/I3bJJ6xtcVO3puI',
    'Content-Type': 'application/json'
  }

  response = requests.request("POST", url, headers=headers, data=payload)
  return response.text

In [43]:
sgt = []
i=0
for global_seg in global_segment_list:
    rep_text = crear_seg([global_seg])
    sgt.append(rep_text)
    #i+=1
    #if i == 3:
    #    break
    time.sleep(20)
sgt

['Customer segments were created: [131737-DAY_Elasticidad2023-02-28_CMX_Frutas & Verduras_TEST_D1]',
 'Customer segments were created: [131612-DAY_Elasticidad2023-02-28_CMX_Frutas & Verduras_CONTROL_D1]',
 'Customer segments were created: [131613-DAY_Elasticidad2023-02-28_CMX_Frutas & Verduras_ALL_D1]']

In [46]:
sgt[0]

['Customer segments were created: [131737-DAY_Elasticidad2023-02-28_CMX_Frutas & Verduras_TEST_D1]',
 'Customer segments were created: [131612-DAY_Elasticidad2023-02-28_CMX_Frutas & Verduras_CONTROL_D1]',
 'Customer segments were created: [131613-DAY_Elasticidad2023-02-28_CMX_Frutas & Verduras_ALL_D1]']

In [51]:
(sgt[2][32:].strip('][')[:6], sgt[2][32:].strip('][')[37:])

('131613', 'Frutas & Verduras_ALL_D1')

In [17]:
segmentos = [i[0] for i in sgt if i[1][i[1].index("_")+1:i[1].index("_", i[1].index("_")+1)] == 'ALL']
subcategorias = [i[1][:i[1].index("_")] for i in sgt if i[1][i[1].index("_")+1:i[1].index("_", i[1].index("_")+1)] == 'ALL']

In [18]:
for i,j in enumerate(segmentos):
    print(str(j)+"\t"+subcategorias[i])

123952	Arroz
123852	Azeites, Óleos e Vinagres
123804	Cervejas
123906	Feijão
123657	Frutas
123908	Legumes
123805	Leite
123706	Queijos
123806	Refrigerantes e Energéticos
